In [44]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score,accuracy_score,confusion_matrix, f1_score, precision_score, auc,roc_auc_score,roc_curve, precision_recall_curve
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import ClusterCentroids,NearMiss, RandomUnderSampler
from imblearn.combine import SMOTEENN,SMOTETomek
#from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import LogisticRegression
#from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [45]:
data = pd.read_csv('data_source/log2.csv')
data.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


In [46]:
labelencoder=LabelEncoder()
for col in ['Source Port','Destination Port','NAT Source Port','NAT Destination Port']:
    data[col] = labelencoder.fit_transform(data[col])
 
data.tail(10)

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
65522,12195,3212,0,0,deny,66,66,0,1,0,1,0
65523,13790,1215,0,0,deny,66,66,0,1,0,1,0
65524,12688,1804,0,0,deny,62,62,0,1,0,1,0
65525,22516,10,14203,9,allow,356,118,238,2,30,1,1
65526,10740,2161,28976,1669,allow,70,70,0,2,8,2,0
65527,21002,15,5390,11,allow,314,192,122,6,15,4,2
65528,10093,15,5499,11,allow,4680740,67312,4613428,4675,77,985,3690
65529,13542,28,0,0,drop,70,70,0,1,0,1,0
65530,13541,28,0,0,drop,70,70,0,1,0,1,0
65531,13538,28,0,0,drop,70,70,0,1,0,1,0


In [47]:
data['Action'].replace({'allow':0, 'deny':1, 'drop':2, 'reset-both':3},inplace=True)

data.tail(10)

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
65522,12195,3212,0,0,1,66,66,0,1,0,1,0
65523,13790,1215,0,0,1,66,66,0,1,0,1,0
65524,12688,1804,0,0,1,62,62,0,1,0,1,0
65525,22516,10,14203,9,0,356,118,238,2,30,1,1
65526,10740,2161,28976,1669,0,70,70,0,2,8,2,0
65527,21002,15,5390,11,0,314,192,122,6,15,4,2
65528,10093,15,5499,11,0,4680740,67312,4613428,4675,77,985,3690
65529,13542,28,0,0,2,70,70,0,1,0,1,0
65530,13541,28,0,0,2,70,70,0,1,0,1,0
65531,13538,28,0,0,2,70,70,0,1,0,1,0


Check magnitude of imbalance

In [48]:
data['Action'].value_counts()

0    37640
1    14987
2    12851
3       54
Name: Action, dtype: int64

Split into training and test datasets

In [49]:
# split the data into train and test set
train, test = train_test_split(data, test_size=0.30, random_state=0,stratify=data['Action'])

In [50]:
X = data.iloc[:, data.columns != 'Action']
y = data.iloc[:, data.columns == 'Action']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0,stratify=data['Action'])

print(X_train.shape)
print(X_test.shape)

(45872, 11)
(19660, 11)


1. SMOTETomek

In [64]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(sampling_strategy='auto')
X_smt, y_smt = smt.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_smt.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_smt,y_smt)

train_yhat = knn.predict(X_smt)
train_f1 = f1_score(y_smt,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_smt,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_smt, y_smt, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
0         26341
2         26340
3         26314
1         26299
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26341
           1       1.00      1.00      1.00     26299
           2       1.00      1.00      1.00     26340
           3       1.00      1.00      1.00     26314

    accuracy                           1.00    105294
   macro avg       1.00      1.00      1.00    105294
weighted avg       1.00      1.00      1.00    105294

Test Score
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     11292
           1       0.99      0.96      0.97      4496
           2       0.98      1.00      0.99      3856
           3       0.04      0.56      0.08        16

    accuracy                           0.98     19660
   macro avg       0.75      0.88      0.76     19660
weighted avg       0.99      0.98      0.99     19660

AVG F1-Score Train: 0.9995725325337793
AVG F1-Score Test: 0.7596210679421458


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.98722636 0.98865093 0.98846099 0.99045539 0.99017001]


2. SMOTEENN

In [63]:
from imblearn.combine import SMOTEENN

sme = SMOTEENN(sampling_strategy='auto')
X_sme, y_sme = sme.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_sme.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_sme,y_sme)

train_yhat = knn.predict(X_sme)
train_f1 = f1_score(y_sme,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_sme,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_sme, y_sme, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
2         26256
0         25836
3         25813
1         25252
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     25836
           1       1.00      1.00      1.00     25252
           2       1.00      1.00      1.00     26256
           3       1.00      1.00      1.00     25813

    accuracy                           1.00    103157
   macro avg       1.00      1.00      1.00    103157
weighted avg       1.00      1.00      1.00    103157

Test Score
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     11292
           1       0.98      0.93      0.95      4496
           2       0.97      1.00      0.99      3856
           3       0.02      0.56      0.05        16

    accuracy                           0.97     19660
   macro avg       0.75      0.87      0.74     19660
weighted avg       0.99      0.97      0.98     19660

AVG F1-Score Train: 1.0
AVG F1-Score Test: 0.7444785431204036


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.99859442 0.99849748 0.99873976 0.99859435 0.99806117]


3. RandomOverSampler

In [62]:
ros = RandomOverSampler(sampling_strategy='not majority')
X_ros, y_ros = ros.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_ros.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_ros,y_ros)

train_yhat = knn.predict(X_ros)
train_f1 = f1_score(y_ros,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_ros,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_ros, y_ros, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
0         26348
1         26348
2         26348
3         26348
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26348
           1       1.00      1.00      1.00     26348
           2       1.00      1.00      1.00     26348
           3       1.00      1.00      1.00     26348

    accuracy                           1.00    105392
   macro avg       1.00      1.00      1.00    105392
weighted avg       1.00      1.00      1.00    105392

Test Score
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     11292
           1       0.98      0.99      0.98      4496
           2       0.98      1.00      0.99      3856
           3       0.33      0.12      0.18        16

    accuracy                           0.99     19660
   macro avg       0.82      0.77      0.79     19660
weighted avg       0.99      0.99      0.99     19660

AVG F1-Score Train: 0.999639441245407
AVG F1-Score Test: 0.7877082023039761


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.99587267 0.99743821 0.9973432  0.99852927 0.9980074 ]


4. SMOTE

In [61]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='not majority')
X_sm, y_sm = sm.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_sm.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_sm,y_sm)

train_yhat = knn.predict(X_sm)
train_f1 = f1_score(y_sm,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_sm,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_sm, y_sm, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
0         26348
1         26348
2         26348
3         26348
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26348
           1       1.00      1.00      1.00     26348
           2       1.00      1.00      1.00     26348
           3       1.00      1.00      1.00     26348

    accuracy                           1.00    105392
   macro avg       1.00      1.00      1.00    105392
weighted avg       1.00      1.00      1.00    105392

Test Score
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     11292
           1       0.99      0.96      0.97      4496
           2       0.98      1.00      0.99      3856
           3       0.05      0.69      0.10        16

    accuracy                           0.98     19660
   macro avg       0.76      0.91      0.76     19660
weighted avg       0.99      0.98      0.99     19660

AVG F1-Score Train: 0.9993926987544641
AVG F1-Score Test: 0.7631384919458867


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.98595759 0.98757057 0.98847139 0.98913559 0.98913559]
